In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 512 #8
batch_size = 4

max_iters = 3000
learning_rate = 3e-3
eval_iters = 50 #500
n_embd = 384
n_head = 4
n_layer = 4


dropout = 0.2 # drop out 20% of neurons. turn to zero to prevent overfitting.

cuda


In [2]:
# read in our corpus into text string
chars = ""
with open('data\philosophers.txt','r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)
print(text[:200])



=== PLATO ===

How you, O Athenians, have been affected by my accusers, I cannot tell;
but I know that they almost made me forget who I was—so persuasively
did they speak; and yet they have hardly 


In [3]:
# encoder and decoder for strings to integers and vice versa

#chars = sorted(set(text))
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
# use first 80% of corpus for training; remaining 20% for validation
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

In [ ]:
class Head(nn.Module):
    """ one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        # computer attention scores ("affinities")

        # head_size = subproblems
        # scaling controls so no one individual head dominates / "hear all voices evenly"
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T, hs) @ (B, hs, T) -> (B,T,T)

        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v # (B, T, T) @ (B,T, hs) -> (B ,T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) #heads running in parallel
        self.proj = nn.Linear(head_size * num_heads, n_embd) # projection
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # concatenate along feature dimension (B,T,F) -> (B,T, [h1, h1, h1, h1, h2,h2,h2,h2,h3,h3,h3,h3])
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity"""
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4* n_embd, n_embd),
            nn.Dropout(dropout) # used to prevent overfitting. look up later.
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """ Transformer block: communication follwoed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head # number of features that each head will capture in multi-head attention
        self.sa = MultiHeadAttention(n_head, head_size) # self-attention
        self.ffwd = FeedForward(n_embd) # feed-forward
        self.ln1 = nn.LayerNorm(n_embd)  # post-norm. 
        self.ln2 = nn.LayerNorm(n_embd) # post-norm. 

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x+y) # add+norm
        y = self.ffwd(x)
        x = self.ln2(x+y) # add+norm
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embeddings_table = nn.Embedding(vocabulary_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) # decoder layers
        self.ln_f = nn.LayerNorm(n_embd) #normalization
        self.lm_head = nn.Linear(n_embd, vocabulary_size) # linear transformation

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, index, targets=None):
        print(f'index: {index}')
        #logits = self.token_embeddings_table(index)  # (B,T,C)
        B, T = index.shape

        tok_emb = self.token_embeddings_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss #return logits.view(B, T, C), loss
    

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(index)          # (B,T,C)
            #print(f'index {index}')
            print(logits)
            logits = logits[:, -1, :]                # take last time step -> (B,C)
            #print('last')
            #print(logits)
            probs = F.softmax(logits, dim=-1)        # turn into probabilities
            #print('probs')
            #print(probs)
            index_next = torch.multinomial(probs, 1) # sample next token -> (B,1)
            #print(index_next)
            index = torch.cat((index, index_next), dim=1)
        return index


In [ ]:
model = GPTLanguageModel(vocab_size)

m = model.to(device)

In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
# create the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    losses = estimate_loss()
    print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model,f)
print('model saved')

In [ ]:
prompt = "Hello! Can you see me?"
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

In [9]:
import inspect
help(nn.Linear)
#print(inspect.signature(nn.Module.__init__))

Help on class Linear in module torch.nn.modules.linear:

class Linear(torch.nn.modules.module.Module)
 |  Linear(in_features: int, out_features: int, bias: bool = True, device=None, dtype=None) -> None
 |  
 |  Applies an affine linear transformation to the incoming data: :math:`y = xA^T + b`.
 |  
 |  This module supports :ref:`TensorFloat32<tf32_on_ampere>`.
 |  
 |  On certain ROCm devices, when using float16 inputs this module will use :ref:`different precision<fp16_on_mi200>` for backward.
 |  
 |  Args:
 |      in_features: size of each input sample
 |      out_features: size of each output sample
 |      bias: If set to ``False``, the layer will not learn an additive bias.
 |          Default: ``True``
 |  
 |  Shape:
 |      - Input: :math:`(*, H_\text{in})` where :math:`*` means any number of
 |        dimensions including none and :math:`H_\text{in} = \text{in\_features}`.
 |      - Output: :math:`(*, H_\text{out})` where all but the last dimension
 |        are the same shap

In [ ]:
class fakename(nn.Linear):
    def __init__(self, name, in_features, out_features):
        super().__init__(in_features, out_features)
        self.name = name

me = fakename("Carlos", 4, 4)

: 